<a href="https://colab.research.google.com/github/Hardik357/CryptoRNN/blob/main/CryptoCurrencyPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing
import pandas as pd
import numpy as np
import random
from collections import deque

In [15]:
seq_len = 60
future_period_predict = 3
ratio_to_predict = "LTC-USD"


def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0


main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]

for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"

    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    
    df.rename(columns={"close": f"{ratio}_close",
                       "volume": f"{ratio}_volume"}, inplace=True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

In [ ]:
print(main_df.head())
for c in main_df.columns:
    print(c)

In [ ]:
main_df['future'] = main_df[f"{ratio_to_predict}_close"].shift(-future_period_predict)
print(main_df[[f"{ratio_to_predict}_close", "future"]].head())

In [ ]:
main_df['target'] = list(map(classify, main_df[f"{ratio_to_predict}_close"], main_df["future"]))
print(main_df[[f"{ratio_to_predict}_close", "future", "target"]].head(10))

In [ ]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05 * len(times))]
print(last_5pct)

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

In [50]:
def preprocess_df(df):
    df = df.drop('future', 1)
    for col in df.columns:
        if col!= "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)

    sequential_data = []
    prev_days = deque(maxlen = seq_len)
    for i in df.values:
        prev_days.append([n for n in i[:,-1]])
        if len(prev_days) == seq_len:
            sequential_data.append([np.array(prev_days)], i[-1])
    random.shuffle(sequential_data)
preprocess_df(main_df)
# train_x, train_y = preprocess_df(main_df)
# validation_x, validation_y = preprocess_df(main_df)

IndexError: ignored